Build df using the following files and save in `../data/data_merged.csv`:
* pdx drug response data
* rna expression
* mordred drug descriptors
* metadata

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
from pathlib import Path
import glob
from pprint import pprint
import pandas as pd
import numpy as np

fdir = Path.cwd()
print(fdir)
sys.path.append(str(fdir/'../src'))
from config import cfg

from build_df import load_rsp, load_rna, load_dd, load_meta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/vol/ml/apartin/projects/pdx-histo/nbs


In [10]:
rsp_dpath = cfg.DATADIR/'studies/pdm/ncipdm_drug_response'
rna_dpath = cfg.DATADIR/'combined_rnaseq_data_lincs1000_combat'
dd_dpath = cfg.DATADIR/'dd.mordred.with.nans'
meta_dpath = cfg.DATADIR/'meta/meta_merged.csv'

In [11]:
# Load
rsp = load_rsp(rsp_dpath)
rna = load_rna(rna_dpath)
dd = load_dd(dd_dpath)
meta = load_meta(meta_dpath)


Dropped 77 duplicates.



In [4]:
# Merge
data = rsp.merge(rna, on='Sample', how='inner')
data = data.merge(dd, left_on='Drug1', right_on='ID', how='inner').reset_index(drop=True)
data = data.merge(meta.drop(columns=['patient_id', 'specimen_id', 'sample_id']), on='Sample', how='inner').reset_index(drop=True)
data = data.sample(frac=1.0, random_state=0, axis=0).reset_index(drop=True)  # TODO is this necessary??

In [5]:
def check_prfx(x):
    return True if x.startswith('ge_') or x.startswith('dd_') else False

# Re-org columns
meta_df = data.drop(columns=[c for c in data.columns if check_prfx(c)])
fea_df = data.drop(columns=meta_df.columns)
data = pd.concat([meta_df, fea_df], axis=1)

In [7]:
data.image_id.nunique()

409